<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Train-test Split and Cross-Validation Lab

_Authors: Joseph Nelson (DC), Kiefer Katovich (SF)_

---

## Review of train/test validation methods

We've discussed overfitting, underfitting, and how to validate the "generalizeability" of your models by testing them on unseen data. 

In this lab you'll practice two related validation methods: 
1. **train/test split**
2. **k-fold cross-validation**

Train/test split and k-fold cross-validation both serve two useful purposes:
- We prevent overfitting by not using all the data, and
- We retain some remaining data to evaluate our model.

In the case of cross-validation, the model fitting and evaluation is performed multiple times on different train/test splits of the data.

Ultimately we can the training and testing validation framework to compare multiple models on the same dataset. This could be comparisons of two linear models, or of completely different models on the same data.


## Instructions

For your independent practice, fit **three different models** on the Boston housing data. For example, you could pick three different subsets of variables, one or more polynomial models, or any other model that you like. 

**Start with train/test split validation:**
* Fix a testing/training split of the data
* Train each of your models on the training data
* Evaluate each of the models on the test data
* Rank the models by how well they score on the testing data set.

**Then try K-Fold cross-validation:**
* Perform a k-fold cross validation and use the cross-validation scores to compare your models. Did this change your rankings?
* Try a few different K-splits of the data for the same models.

If you're interested, try a variety of response variables.  We start with **MEDV** (the `.target` attribute from the dataset load method).

In [1]:
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston

boston = load_boston()

In [10]:
type(boston)

sklearn.utils.Bunch

In [9]:
dir(boston)

#dataset['target'] = 1D numpy array of target attribute values
# dataset['data'] = 2D numpy array of attribute values
# dataset['feature_names'] - 1D numpy array of names of the attributes
# dataset['DESCR'] = text description of the dataset

['DESCR', 'data', 'feature_names', 'filename', 'target']

In [16]:
bos_df = pd.DataFrame(np.c_[boston['data'], boston['target']],
                  columns= np.append(boston['feature_names'], ['target']))

#np.c_ = concatenate

In [17]:
bos_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### 1. Clean up any data problems

Load the Boston housing data.  Fix any problems, if applicable.

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in $1000's

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/

In [29]:
bos_df.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
target     0
dtype: int64

In [31]:
bos_df.dtypes

CRIM       float64
ZN         float64
INDUS      float64
CHAS       float64
NOX        float64
RM         float64
AGE        float64
DIS        float64
RAD        float64
TAX        float64
PTRATIO    float64
B          float64
LSTAT      float64
target     float64
dtype: object

In [23]:
bos_df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [27]:
bos_df.CRIM.sort_values()

0       0.00632
284     0.00906
285     0.01096
341     0.01301
55      0.01311
54      0.01360
195     0.01381
57      0.01432
194     0.01439
348     0.01501
283     0.01501
256     0.01538
353     0.01709
200     0.01778
347     0.01870
64      0.01951
286     0.01965
204     0.02009
56      0.02055
202     0.02177
193     0.02187
342     0.02498
343     0.02543
2       0.02729
1       0.02731
39      0.02763
93      0.02875
349     0.02899
5       0.02985
337     0.03041
         ...   
477    15.02340
437    15.17720
376    15.28800
468    15.57570
425    15.86030
381    15.87440
385    16.81180
379    17.86670
415    18.08460
374    18.49820
412    18.81100
375    19.60910
384    20.08490
406    20.71620
440    22.05110
387    22.59710
378    23.64820
386    24.39380
403    24.80170
400    25.04610
417    25.94060
413    28.65580
427    37.66190
398    38.35180
404    41.52920
414    45.74610
410    51.13580
405    67.92080
418    73.53410
380    88.97620
Name: CRIM, Length: 506,

In [36]:
bos_df.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'target'],
      dtype='object')

In [38]:
feature_cols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT']
X = bos_df[feature_cols]
y = bos_df.target

### 2. Select 3-4 variables with your dataset to perform a 50/50 test train split on

- Use sklearn.
- Score and plot your predictions.

In [49]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5, random_state=123)

In [50]:
# Before splitting
print(X.shape)

# After splitting
print(X_train.shape)
print(X_test.shape)

(506, 13)
(253, 13)
(253, 13)


In [48]:
#Create benchmark MSE

# Create a NumPy array with the same shape as y_test.
y_null = np.zeros_like(y_test, dtype=float)
# Fill the array with the mean value of y_test.
y_null.fill(y_test.mean())
y_null
# Compute null RMSE.
np.sqrt(metrics.mean_squared_error(y_test, y_null))

8.884944468283209

In [52]:
#Compute MSE of baseline

# Import the class.
from sklearn.linear_model import LinearRegression

# Instantiate the model.
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
base_MSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

base_MSE

4.925137187354487

In [58]:
#Function for running MSE
def train_test_rmse(df, feature_cols, trainsize):
    X = df[feature_cols]
    y = df.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=trainsize, random_state=123)
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    return np.sqrt(metrics.mean_squared_error(y_test, y_pred))


In [59]:
train_test_rmse(bos_df, ['CRIM', 'ZN', 'INDUS'], 0.5)

7.852709732346094

### 3. Try 70/30 and 90/10
- Score and plot.  
- How do your metrics change?

In [60]:
train_test_rmse(bos_df, ['CRIM', 'ZN', 'INDUS'], 0.7)

7.874311535786484

In [61]:
train_test_rmse(bos_df, ['CRIM', 'ZN', 'INDUS'], 0.9)

10.561286565052685

### 4. Try K-Folds cross-validation with K between 5-10 for your regression. 

- What seems optimal? 
- How do your scores change?  
- What the variance of scores like?
- Try different folds to get a sense of how this impacts your score.

In [104]:
from sklearn import model_selection
kf = model_selection.KFold(n_splits=5, shuffle=True)

kf

KFold(n_splits=5, random_state=None, shuffle=True)

In [95]:
X.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')

In [96]:
X2 = bos_df[['CRIM', 'ZN', 'INDUS']]
X2.columns

Index(['CRIM', 'ZN', 'INDUS'], dtype='object')

In [97]:
mse_values = []
scores = []
n = 0

print("~~~~ CROSS VALIDATION each fold ~~~~")
for train_index, test_index in kf.split(X2, y):
    lr = LinearRegression().fit(X2.iloc[train_index], y.iloc[train_index])
    
    mse_values.append(np.sqrt(metrics.mean_squared_error(y.iloc[test_index], lr.predict(X2.iloc[test_index]))))
    scores.append(lr.score(X2, y))
#    print(test_index)
    
    n += 1
    print('Model {}'.format(n))
    print('MSE: {}'.format(mse_values[n-1]))
    print('R2: {}\n'.format(scores[n-1]))


print("~~~~ SUMMARY OF CROSS VALIDATION ~~~~")
print('Mean of MSE for all folds: {}'.format(np.mean(mse_values)))
print('Mean of R2 for all folds: {}'.format(np.mean(scores)))

~~~~ CROSS VALIDATION each fold ~~~~
Model 1
MSE: 8.442050745316111
R2: 0.29263428208115694

Model 2
MSE: 7.610977761396109
R2: 0.29279165757458947

Model 3
MSE: 8.066407738133632
R2: 0.29335961019764445

Model 4
MSE: 6.861269499056838
R2: 0.29234702899422116

Model 5
MSE: 7.722245485463072
R2: 0.2934782376919609

~~~~ SUMMARY OF CROSS VALIDATION ~~~~
Mean of MSE for all folds: 7.740590245873152
Mean of R2 for all folds: 0.2929221633079146


In [114]:
from sklearn.model_selection import cross_val_score
print(np.sqrt(np.mean(-cross_val_score(lr, X2, y, cv=kf, scoring='neg_mean_squared_error'))))
print(np.mean(cross_val_score(lr, X2, y, cv=kf)))
#accomplishes the same thing as function above

7.799139911285693
0.2616501538217523


### 5. [Bonus] optimize the $R^2$ score

Can you optimize your R^2 by selecting the best features and validating the model using either train/test split or K-Folds?

Your code will need to iterate through the different combinations of predictors, cross-validate the current model parameterization, and determine which set of features performed best.

The number of K-folds is up to you.

> *Hint:* the `itertools` package is useful for combinations and permutations.


In [74]:
from itertools import *

In [177]:
combos = []
scores = []
for i in combinations(X.columns, 2):
    combos.append(list(i))
    scores.append(np.mean(cross_val_score(lr, bos_df[list(i)], y, cv=kf)))
    test_df = pd.DataFrame(pd.Series(combos))
    test_df['scores'] = pd.Series(scores)
    test_df.columns = ['combo','score']
test_df.tail()


,combo,score
73,"[TAX, B]",0.214666
74,"[TAX, LSTAT]",0.543231
75,"[PTRATIO, B]",0.295778
76,"[PTRATIO, LSTAT]",0.600424
77,"[B, LSTAT]",0.540303


In [176]:
test_df.sort_values(by='score',ascending=True).head()

,combo,score
36,"[CHAS, DIS]",0.084177
17,"[ZN, DIS]",0.111126
66,"[DIS, B]",0.115313
6,"[CRIM, DIS]",0.115590
40,"[CHAS, B]",0.121847


In [186]:
def combinator(dataframe,n):
    combination_list = []
    x = 2
    while x < n:
        combination_list.append(combinations(X.columns, x))
    x += 1
    return combination_list

# Not Working!

In [188]:
combinator(X,3)

KeyboardInterrupt: 

### 5.1 Can you explain what could be wrong with this approach?

In [9]:
# A:

# ????
# The number of combinations/permutations is far too large. If we ran it for combinations of 7 x our 13 factors, that's 1,716--the peak for a combination. This kind of blind assessment also devalues considering relationships and examining correlations.
# It doesn't include squared permutations of the second order?
# R^2 < RMSE ?
# And how do you set train size with k-folds??

### 6. [Bonus] Explore another target variable and practice `patsy` formulas

Can you find another response variable, given a combination of predictors, that can be predicted accurately through the exploration of different predictors in this dataset?

**Try out using patsy to construct your target and predictor matrices from formula strings.**

> *Tip: Check out pairplots, coefficients, and pearson scores.*

In [10]:
import patsy

# A:

# ???? want to do it without patsy